## Library Import

In [ ]:
import pandas as pd
import re
import nltk
import torch
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.nn import Embedding
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import random
import torch.optim as optim
from nltk import pos_tag
import torch.nn.functional as F
!pip install banglanltk
!pip install rouge
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 kB 4.2 MB/s eta 0:00:00
Mounted at /content/drive


## Dataset Import

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/CSE400 Dataset/english_To_Bangla (1).csv')
data.head()

,Summary,Translated_Summary
0,India markets regulator ups scrutiny of IPO do...,ভারতের বাজার নিয়ন্ত্রক আইপিও নথির যাচাই-বাছাই...
1,IMF's Georgieva 'very confident' on soft landi...,আইএমএফের জর্জিভা নরম অবতরণে 'খুব আত্মবিশ্বাসী'...
2,UK employers plan smaller pay rises for 2024: ...,UK নিয়োগকারীরা 2024-এর জন্য ছোট বেতন বৃদ্ধির ...
3,"EU agrees on looser fiscal rules to cut debt, ...","EU ঋণ কমাতে, বিনিয়োগ বাড়াতে শিথিল আর্থিক নিয..."
4,US imposes sanctions for violations of Russia ...,রাশিয়ার তেলের মূল্যসীমা লঙ্ঘনের জন্য যুক্তরাষ...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45149 entries, 0 to 45148
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Summary             45149 non-null  object
 1   Translated_Summary  45149 non-null  object
dtypes: object(2)
memory usage: 705.6+ KB


## Data Preprocessing

### Text Cleaning

In [ ]:
import banglanltk as bn
def clean_text(text):
  text = re.sub(r"<.*?>", "", text)
  text = re.sub(r"[^\w\s]", "", text)
  text = text.lower()
  text = " ".join(text.split())
  return text

data["Summary"] = data["Summary"].apply(clean_text)

data['Translated_Summary'] = data['Translated_Summary'].apply(bn.clean_text)

data.head()

,Summary,Translated_Summary
0,india markets regulator ups scrutiny of ipo do...,ভারতের বাজার নিয়ন্ত্রক আইপিও নথির যাচাইবাছাই ...
1,imfs georgieva very confident on soft landing ...,আইএমএফের জর্জিভা নরম অবতরণে খুব আত্মবিশ্বাসী র...
2,uk employers plan smaller pay rises for 2024 c...,UK নিয়োগকারীরা 2024এর জন্য ছোট বেতন বৃদ্ধির প...
3,eu agrees on looser fiscal rules to cut debt b...,EU ঋণ কমাতে বিনিয়োগ বাড়াতে শিথিল আর্থিক নিয়...
4,us imposes sanctions for violations of russia ...,রাশিয়ার তেলের মূল্যসীমা লঙ্ঘনের জন্য যুক্তরাষ...


### Tokenization

In [ ]:
nltk.download('punkt')

data["Summary"] = data["Summary"].apply(word_tokenize)
data['Translated_Summary'] = data['Translated_Summary'].apply(bn.word_tokenize)

data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,Summary,Translated_Summary
0,"[india, markets, regulator, ups, scrutiny, of,...","[ভারতের, বাজার, নিয়ন্ত্রক, আইপিও, নথির, যাচাই..."
1,"[imfs, georgieva, very, confident, on, soft, l...","[আইএমএফের, জর্জিভা, নরম, অবতরণে, খুব, আত্মবিশ্..."
2,"[uk, employers, plan, smaller, pay, rises, for...","[UK, নিয়োগকারীরা, 2024এর, জন্য, ছোট, বেতন, বৃ..."
3,"[eu, agrees, on, looser, fiscal, rules, to, cu...","[EU, ঋণ, কমাতে, বিনিয়োগ, বাড়াতে, শিথিল, আর্থ..."
4,"[us, imposes, sanctions, for, violations, of, ...","[রাশিয়ার, তেলের, মূল্যসীমা, লঙ্ঘনের, জন্য, যু..."


### Removing Stopwords

In [ ]:
nltk.download('stopwords')

stop_words = stopwords.words("english")
data["Summary"] = data["Summary"].apply(lambda x: [token for token in x if token not in stop_words])

with open("/content/drive/MyDrive/CSE400 Dataset/ranksnl-bengali.txt", "r", encoding="utf-8") as f:
  stopwords = [line.strip() for line in f]

def remove_stopwords(text_list, stopwords):
    filtered_text = [word for word in text_list if word not in stopwords]
    return filtered_text

data["Translated_Summary"] = data["Translated_Summary"].apply(remove_stopwords, stopwords=stopwords)

data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Summary,Translated_Summary
0,"[india, markets, regulator, ups, scrutiny, ipo...","[ভারতের, বাজার, নিয়ন্ত্রক, আইপিও, নথির, যাচাই..."
1,"[imfs, georgieva, confident, soft, landing, se...","[আইএমএফের, জর্জিভা, নরম, অবতরণে, আত্মবিশ্বাসী,..."
2,"[uk, employers, plan, smaller, pay, rises, 202...","[UK, নিয়োগকারীরা, 2024এর, ছোট, বেতন, বৃদ্ধির,..."
3,"[eu, agrees, looser, fiscal, rules, cut, debt,...","[EU, ঋণ, কমাতে, বিনিয়োগ, বাড়াতে, শিথিল, আর্থ..."
4,"[us, imposes, sanctions, violations, russia, o...","[রাশিয়ার, তেলের, মূল্যসীমা, লঙ্ঘনের, যুক্তরাষ..."


### Lemmatization

In [ ]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

data["Summary"] = data["Summary"].apply(lemmatize_tokens)

data.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...


,Summary,Translated_Summary
0,"[india, market, regulator, ups, scrutiny, ipo,...","[ভারতের, বাজার, নিয়ন্ত্রক, আইপিও, নথির, যাচাই..."
1,"[imf, georgieva, confident, soft, landing, see...","[আইএমএফের, জর্জিভা, নরম, অবতরণে, আত্মবিশ্বাসী,..."
2,"[uk, employer, plan, smaller, pay, rise, 2024,...","[UK, নিয়োগকারীরা, 2024এর, ছোট, বেতন, বৃদ্ধির,..."
3,"[eu, agrees, looser, fiscal, rule, cut, debt, ...","[EU, ঋণ, কমাতে, বিনিয়োগ, বাড়াতে, শিথিল, আর্থ..."
4,"[u, imposes, sanction, violation, russia, oil,...","[রাশিয়ার, তেলের, মূল্যসীমা, লঙ্ঘনের, যুক্তরাষ..."


### Vocabulary Creation

In [ ]:
summary_vocab = build_vocab_from_iterator(data["Summary"], specials=["<pad>", "<sos>", "<eos>", "<unk>"])
translation_vocab = build_vocab_from_iterator(data["Translated_Summary"], specials=["<pad>", "<sos>", "<eos>", "<unk>"])

summary_itos = summary_vocab.get_itos()
summary_stoi = summary_vocab.get_stoi()
translation_itos = translation_vocab.get_itos()
translation_stoi = translation_vocab.get_stoi()

### Padding

In [ ]:
summary_max_len = int(data['Summary'].str.len().quantile(0.70))
translation_max_len = int(data['Translated_Summary'].str.len().quantile(0.70))

print("95th percentile length for Summary:", summary_max_len)
print("95th percentile length for Translation:", translation_max_len)

95th percentile length for Summary: 18
95th percentile length for Translation: 18


In [ ]:
def pad_tokens(tokens, vocab_stoi, max_len):
    pad_index = vocab_stoi["<pad>"]
    tokens = tokens[:max_len]
    sequence = [vocab_stoi.get(token, vocab_stoi["<unk>"]) for token in tokens] + [pad_index] * (max_len - len(tokens))
    return sequence

summary_sequences = [pad_tokens(tokens, summary_stoi, 18) for tokens in data["Summary"]]
translation_sequences = [pad_tokens(tokens, translation_stoi, 18) for tokens in data["Translated_Summary"]]

summary_tensor = torch.tensor(summary_sequences)
translation_tensor = torch.tensor(translation_sequences)

## Word Vectorization

In [ ]:
glove_vectors = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/CSE400 Dataset/glove.6B.300d.txt",  no_header=True)
summary_vocab_size = len(summary_vocab)
summary_embedding_dim = 300

english_pretrained = torch.zeros(summary_vocab_size, summary_embedding_dim)
for word, i in summary_vocab.get_stoi().items():
    if word in glove_vectors.key_to_index:
        english_pretrained[i] = torch.tensor(glove_vectors[word])
    else:
        english_pretrained[i] = torch.randn(summary_embedding_dim)
pad_idx = summary_vocab.get_stoi()["<pad>"]
english_pretrained[pad_idx] = torch.zeros(summary_embedding_dim)
english_pretrained.requires_grad = False

summary_embedding = Embedding.from_pretrained(english_pretrained, freeze=True)

In [ ]:
bangla_glove = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/CSE400 Dataset/bn_glove.39M.300d.txt",  no_header=True)
translation_vocab_size = len(translation_vocab)
trasnlation_embedding_dim = 300

bangla_pretrained = torch.zeros(translation_vocab_size, trasnlation_embedding_dim)
for word, i in translation_vocab.get_stoi().items():
    if word in bangla_glove.key_to_index:
        bangla_pretrained[i] = torch.tensor(bangla_glove[word])
    else:
        bangla_pretrained[i] = torch.randn(trasnlation_embedding_dim)
pad_idx = translation_vocab.get_stoi()["<pad>"]
bangla_pretrained[pad_idx] = torch.zeros(trasnlation_embedding_dim)
bangla_pretrained.requires_grad = False

translation_embedding = Embedding.from_pretrained(bangla_pretrained, freeze=True)

## Train Test Split and Dataset Loader

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(summary_tensor, translation_tensor, test_size=0.1, random_state=42)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, summaries, translations):
        self.summaries = summaries
        self.translations = translations

    def __len__(self):
        return len(self.summaries)

    def __getitem__(self, idx):
        return self.summaries[idx], self.translations[idx]

train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

## Seq2Seq Bahdanau Attention

In [ ]:
print("Summary Vocabulary Size: ", len(summary_vocab))
print("Translation Vocabulary Size:", len(translation_vocab))
SOS_token = summary_stoi["<sos>"]
PAD_token = summary_stoi["<pad>"]
EOS_token = summary_stoi["<eos>"]
UNK_token = summary_stoi["<unk>"]

Summary Vocabulary Size:  38954
Translation Vocabulary Size: 53933


In [ ]:
embedding_dim = 300
hidden_dim = 512
output_dim = len(translation_vocab)
n_layers = 2
dropout = 0.30
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### English To Bangla Seq2Seq

In [ ]:
class Seq2SeqModel(nn.Module):
  def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers, dropout, summary_embedding, translation_embedding):
    super().__init__()

    self.output_dim = output_dim
    self.english_embedding = summary_embedding
    self.bangla_embedding = translation_embedding
    self.encoder = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, dropout=dropout)
    self.decoder = nn.LSTM(embedding_dim + hidden_dim, hidden_dim, num_layers=n_layers, dropout=dropout)
    self.W1 = nn.Linear(hidden_dim, hidden_dim)
    self.W2 = nn.Linear(hidden_dim, hidden_dim)
    self.V = nn.Linear(hidden_dim, 1)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, source, target, teacher_forcing_ratio=0.5):
    batch_size = source.shape[0]
    target_len = target.shape[1]
    target_vocab_size = self.output_dim

    embedded =  self.english_embedding(source.transpose(0, 1))
    encoder_outputs, (hidden, cell) = self.encoder(embedded)
    decoder_input = torch.tensor([[SOS_token] * batch_size], device=device)
    decoder_hidden = hidden
    decoder_cell = cell

    outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(device)
    attentions = torch.zeros(batch_size, target_len, source.shape[1]).to(device)

    for t in range(target_len):
      embedded = self.bangla_embedding(decoder_input)
      decoder_hidden_last = decoder_hidden[-1].unsqueeze(0).expand_as(encoder_outputs)

      score = self.V(torch.tanh(self.W1(encoder_outputs) + self.W2(decoder_hidden_last)))
      attention_weights = F.softmax(score, dim=1)
      context_vector = torch.bmm(attention_weights.permute(1, 2, 0), encoder_outputs.permute(1, 0, 2))
      context_vector = context_vector.permute(1, 0, 2)
      embedded_with_context = torch.cat((embedded, context_vector), dim=2)

      decoder_output, (decoder_hidden, decoder_cell) = self.decoder(embedded_with_context, (decoder_hidden, decoder_cell))

      output = self.fc(decoder_output.squeeze(0))
      outputs[:, t, :] = output
      attentions[:, t, :] =  attention_weights.squeeze(-1).transpose(0, 1)
      teacher_force = random.random() < teacher_forcing_ratio
      top1 = output.argmax(1)
      decoder_input = (target[:, t] if teacher_force else top1).unsqueeze(0)
    return outputs, attentions

Bidirectional Implementation

In [ ]:
class Seq2SeqModel(nn.Module):
  def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers, dropout, summary_embedding, translation_embedding):
    super().__init__()

    self.output_dim = output_dim
    self.english_embedding = summary_embedding
    self.bangla_embedding = translation_embedding
    self.encoder = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, dropout=dropout, bidirectional = True)
    self.decoder = nn.LSTM(embedding_dim + 2 * hidden_dim, 2 * hidden_dim, num_layers=n_layers, dropout=dropout)
    self.W1 = nn.Linear(2 * hidden_dim, 2 * hidden_dim)
    self.W2 = nn.Linear(2 * hidden_dim, 2 * hidden_dim)
    self.V = nn.Linear(2 * hidden_dim, 1)
    self.fc = nn.Linear(2 * hidden_dim, output_dim)

  def forward(self, source, target, teacher_forcing_ratio=0.5):
    batch_size = source.shape[0]
    target_len = target.shape[1]
    target_vocab_size = self.output_dim

    embedded =  self.english_embedding(source.transpose(0, 1))
    encoder_outputs, (hidden, cell) = self.encoder(embedded)

    hidden = hidden.view(n_layers, 2, batch_size, hidden_dim)
    cell = cell.view(n_layers, 2, batch_size, hidden_dim)

    decoder_input = torch.tensor([[SOS_token] * batch_size], device=device)

    decoder_hidden = torch.cat((hidden[:, 0, :, :], hidden[:, 1, :, :]), dim=2)
    decoder_cell = torch.cat((cell[:, 0, :, :], cell[:, 1, :, :]), dim=2)

    outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(device)
    attentions = torch.zeros(batch_size, target_len, source.shape[1]).to(device)

    for t in range(target_len):
      embedded = self.bangla_embedding(decoder_input)
      decoder_hidden_last = decoder_hidden[-1].unsqueeze(0).expand_as(encoder_outputs)

      score = self.V(torch.tanh(self.W1(encoder_outputs) + self.W2(decoder_hidden_last)))
      attention_weights = F.softmax(score, dim=1)
      context_vector = torch.bmm(attention_weights.permute(1, 2, 0), encoder_outputs.permute(1, 0, 2))
      context_vector = context_vector.permute(1, 0, 2)
      embedded_with_context = torch.cat((embedded, context_vector), dim=2)

      decoder_output, (decoder_hidden, decoder_cell) = self.decoder(embedded_with_context, (decoder_hidden, decoder_cell))

      output = self.fc(decoder_output.squeeze(0))
      outputs[:, t, :] = output
      attentions[:, t, :] =  attention_weights.squeeze(-1).transpose(0, 1)
      teacher_force = random.random() < teacher_forcing_ratio
      top1 = output.argmax(1)
      decoder_input = (target[:, t] if teacher_force else top1).unsqueeze(0)
    return outputs, attentions

In [ ]:
model = Seq2SeqModel(embedding_dim, hidden_dim, output_dim, n_layers, dropout, summary_embedding, translation_embedding).to(device)

In [ ]:
criterion =  nn.CrossEntropyLoss(ignore_index=PAD_token)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for i, (source, target) in enumerate(train_loader):
        source = source.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        outputs, attentions = model(source, target, teacher_forcing_ratio=0.5)
        output_dim = outputs.shape[-1]
        outputs = outputs.view(-1, output_dim)
        target = target.view(-1)
        loss = criterion(outputs, target)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        if i % 636 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.3f}")

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/CSE400 Dataset/bi_translation_e2b_v1.pth')

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/CSE400 Dataset/bi_translation_e2b.pth'))

## Model Evaluation

In [ ]:
model.eval()

Seq2SeqModel(
  (english_embedding): Embedding(38954, 300)
  (bangla_embedding): Embedding(53933, 300)
  (encoder): LSTM(300, 512, num_layers=2, dropout=0.3, bidirectional=True)
  (decoder): LSTM(1324, 1024, num_layers=2, dropout=0.3)
  (W1): Linear(in_features=1024, out_features=1024, bias=True)
  (W2): Linear(in_features=1024, out_features=1024, bias=True)
  (V): Linear(in_features=1024, out_features=1, bias=True)
  (fc): Linear(in_features=1024, out_features=53933, bias=True)
)

English to Bangla

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from rouge import Rouge
def calculate_scores(references, predictions):
    rouge = Rouge()
    bleu_scores = []
    rouge_scores = {'rouge-1': {'F-1 Score': [], 'Precision': [], 'Recall': []},
                    'rouge-2': {'F-1 Score': [], 'Precision': [], 'Recall': []},
                    'rouge-l': {'F-1 Score': [], 'Precision': [], 'Recall': []}}
    smoothie = SmoothingFunction().method4
    for ref, pred in zip(references, predictions):
        bleu_score = sentence_bleu([ref], pred, weights=(0.8, 0.2, 0, 0), smoothing_function=smoothie)
        bleu_scores.append(bleu_score)
        rouge_score = rouge.get_scores(' '.join(pred), ' '.join(ref), avg=True)
        for key in rouge_scores.keys():
            for metric, label in zip(['f', 'p', 'r'], ['F-1 Score', 'Precision', 'Recall']):
                rouge_scores[key][label].append(rouge_score[key][metric])
    return bleu_scores, rouge_scores


references = []
predictions = []

with torch.no_grad():
    for summaries, translations in test_loader:
        summaries = summaries.to(device)
        translations = translations.to(device)
        outputs, _ = model(summaries, translations)
        predicted_indices = outputs.argmax(dim=-1)
        for i in range(predicted_indices.shape[0]):
            references.append([translation_itos[idx] for idx in translations[i]])
            prediction = []
            for idx in predicted_indices[i]:
                if idx == translation_stoi["<eos>"]:
                    break
                prediction.append(translation_itos[idx])
            if prediction[-1] != "<eos>":
                prediction.append("<eos>")
            predictions.append(prediction)

bleu_scores, rouge_scores = calculate_scores(references, predictions)

print("Average BLEU score:", sum(bleu_scores) / len(bleu_scores))
print("Average ROUGE scores:")
for key in rouge_scores.keys():
    print(f"{key}:")
    for label in ['F-1 Score', 'Precision', 'Recall']:
      print(f"  {label}: {sum(rouge_scores[key][label]) / len(rouge_scores[key][label])}")

Average BLEU score: 0.11148369860922003
Average ROUGE scores:
rouge-1:
  F-1 Score: 0.19934981305156063
  Precision: 0.2185796292212995
  Recall: 0.18905301952008832
rouge-2:
  F-1 Score: 0.03259122491068795
  Precision: 0.031368717198305085
  Recall: 0.0345548179156742
rouge-l:
  F-1 Score: 0.17737118767659096
  Precision: 0.19445481927224958
  Recall: 0.1683312247216861


## Generate Translation

In [ ]:
def generate_translation(input_text):
    tokens = word_tokenize(input_text)
    sequence = pad_tokens(tokens, summary_stoi, 18)
    input_tensor = torch.tensor(sequence).unsqueeze(0).to(device)
    batch_size = input_tensor.shape[0]
    decoder_input = torch.tensor([SOS_token] * batch_size).unsqueeze(1).to(device)
    output_indices = []
    attentions = []
    for _ in range(18):
        with torch.no_grad():
            outputs, attention = model(input_tensor, decoder_input)
        predicted_indices = outputs[:, -1, :].argmax(dim=-1).unsqueeze(1)
        output_indices.append(predicted_indices)
        attentions.append(attention)
        decoder_input = torch.cat((decoder_input, predicted_indices), dim=1)
        if predicted_indices[0][0] == EOS_token:
            break
    output_indices = torch.cat(output_indices, dim=1)
    predicted_words = []
    for index in output_indices[0]:
        if index == SOS_token or index == PAD_token:
            continue
        elif index == EOS_token:
            break
        elif index == UNK_token:
            predicted_words.append("")
        else:
            predicted_words.append(translation_itos[index])
    summary = ' '.join(predicted_words)
    return summary

iter_test_loader = iter(test_loader)
for i in range(25):
  first_batch = next(iter_test_loader)
X_test, y_test = first_batch[0][0], first_batch[1][0]
X_test_tokens = [summary_itos[idx] for idx in X_test if summary_itos[idx] != "<pad>"]
y_test_tokens = [translation_itos[idx] for idx in y_test if translation_itos[idx] != "<pad>"]
input_text = ' '.join(X_test_tokens)
contents = ' '.join(y_test_tokens)
print("Input Text: " + input_text)
print("Sample Translation: " + contents)
print("Generated Translation: " + generate_translation(input_text))